In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!pip3 install -U git+https://github.com/PYFTS/pyFTS
#!pip3 install SimpSOM
from keras.models import Model
from keras.layers import Dense, Input
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from pyFTS.benchmarks import Measures
import statistics
from pyFTS.common.Util import persist_obj, load_obj
from pyFTS.models.multivariate import common, variable, wmvfts
from pyFTS.partitioners import Grid
from pyFTS.common import Membership
import math
import pyFTS
import datetime
from pyFTS.common import Util

In [2]:
df = pd.read_csv('/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/energydata_complete.csv')
df.pop('date')
df.pop('rv1')
df.pop('rv2')

0        13.275433
1        18.606195
2        28.642668
3        45.410389
4        10.084097
           ...    
19730    43.096812
19731    49.282940
19732    29.199117
19733     6.322784
19734    34.118851
Name: rv2, Length: 19735, dtype: float64

In [3]:
x = [1,2,3]
y = [1,2,3]
Measures.rmse(x,y)
x

[1, 2, 3]

In [4]:
def forecast_reducted(train, test, epochs):
    
    input_layer = Input(shape=(26, ))
    encoder_layer_1 = Dense(12, activation="tanh",activity_regularizer=regularizers.l1(10e-5))(input_layer)
    encoder_layer_2 = Dense(2, activation="tanh")(encoder_layer_1)

    decoder_layer_1 = Dense(12, activation="tanh",activity_regularizer=regularizers.l1(10e-5))(encoder_layer_2)
    decoder_layer_2 = Dense(26, activation="tanh")(decoder_layer_1)

    autoencoder = Model(input_layer, decoder_layer_2)
    autoencoder.compile(optimizer = 'adam', loss='mse')
    
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(train)
    X_test = scaler.fit_transform(test)
    autoencoder.fit(x=X_train, y=X_train, epochs=epochs)

    encoder = Model(inputs=input_layer, outputs=encoder_layer_2)

    encoded_train = pd.DataFrame(encoder.predict(X_train))
    encoded_train.columns = ['factor_1', 'factor_2']
    encoded_train = encoded_train.join(train['Appliances'].reset_index())
    
    encoded_test = pd.DataFrame(encoder.predict(X_test))
    encoded_test.columns = ['factor_1', 'factor_2']
    encoded_test = encoded_test.join(test['Appliances'].reset_index())
    

    VX1= variable.Variable("factor_1", data_label="factor_1", partitioner = Grid.GridPartitioner, npart=50,     data=encoded_train )
    VX2= variable.Variable("factor_2", data_label="factor_2", partitioner = Grid.GridPartitioner, npart=50,     data=encoded_train )
    VY = variable.Variable("Appliances", data_label="Appliances", partitioner   = Grid.GridPartitioner, npart=50,   data=encoded_train ) 

    model = pyFTS.models.multivariate.wmvfts.WeightedMVFTS(explanatory_variables=[VX1, VX2, VY], target_variable=VY)
    model.fit(encoded_train)
    forecasts = model.predict(encoded_test)



    return forecasts


In [5]:
forecasts = []
appliances = []
result = {
     "window": [],
     "rmse": [],
     "mape": [],
     "smape": []
}

tam = len(df)
n_windows = 30
windows_length = math.floor(tam / n_windows)
for ct, ttrain, ttest in Util.sliding_window(df, windows_length, 0.75, inc=1):
    if len(ttest) > 0:
        appliance = ttest['Appliances'].reset_index()
        #for
        print('-' * 20)
        print(f'training window {(ct)}')
        forecast = forecast_reducted(ttrain, ttest, 50)
        
        print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
        appliance.pop('index')
        appliances = [i[0] for i in appliance.values]
        rmse = Measures.rmse(appliances,forecast)
        mape = Measures.mape(appliances,forecast)
        smape = Measures.smape(appliances,forecast)
        
        forecasts.append(forecast)
        appliances.append(appliances)
        
        result["rmse"].append(rmse)
        result["mape"].append(mape)
        result["smape"].append(smape)
        result["window"].append(ct)

measures = pd.DataFrame(result)



--------------------
training window 0
Epoch 1/50
16/16 [==============================] - 1s 1ms/step - loss: 0.2423
Epoch 2/50
16/16 [==============================] - 0s 2ms/step - loss: 0.2156
Epoch 3/50
16/16 [==============================] - 0s 1ms/step - loss: 0.1760
Epoch 4/50
16/16 [==============================] - 0s 1ms/step - loss: 0.1341
Epoch 5/50
16/16 [==============================] - 0s 1ms/step - loss: 0.1037
Epoch 6/50
16/16 [==============================] - 0s 2ms/step - loss: 0.0766
Epoch 7/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0613
Epoch 8/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0534
Epoch 9/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0481
Epoch 10/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0457
Epoch 11/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0456
Epoch 12/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0438
Epoch 13/5

16/16 [==============================] - 0s 915us/step - loss: 0.3018
Epoch 2/50
16/16 [==============================] - 0s 1ms/step - loss: 0.2633
Epoch 3/50
16/16 [==============================] - 0s 1ms/step - loss: 0.2433
Epoch 4/50
16/16 [==============================] - 0s 1ms/step - loss: 0.2058
Epoch 5/50
16/16 [==============================] - 0s 1ms/step - loss: 0.1648
Epoch 6/50
16/16 [==============================] - 0s 1ms/step - loss: 0.1302
Epoch 7/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0995
Epoch 8/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0827
Epoch 9/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0724
Epoch 10/50
16/16 [==============================] - 0s 2ms/step - loss: 0.0637
Epoch 11/50
16/16 [==============================] - 0s 2ms/step - loss: 0.0608
Epoch 12/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0604
Epoch 13/50
16/16 [==============================] - 0s 1m

/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/models/multivariate/mvfts.py:165: RuntimeWarning: invalid value encountered in double_scalars
  ret.append(np.dot(mv,mp.T)/np.nansum(mv))


[ 22:27:06] getting statistics
--------------------
training window 2628
Epoch 1/50
16/16 [==============================] - 0s 928us/step - loss: 0.3208
Epoch 2/50
16/16 [==============================] - 0s 1ms/step - loss: 0.2494
Epoch 3/50
16/16 [==============================] - 0s 1ms/step - loss: 0.1831
Epoch 4/50
16/16 [==============================] - 0s 1ms/step - loss: 0.1425
Epoch 5/50
16/16 [==============================] - 0s 1ms/step - loss: 0.1106
Epoch 6/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0902
Epoch 7/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0765
Epoch 8/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0686
Epoch 9/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0625
Epoch 10/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0579
Epoch 11/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0542
Epoch 12/50
16/16 [==============================] - 0

16/16 [==============================] - 0s 863us/step - loss: 0.3399
Epoch 2/50
16/16 [==============================] - 0s 1ms/step - loss: 0.2341
Epoch 3/50
16/16 [==============================] - 0s 1ms/step - loss: 0.2031
Epoch 4/50
16/16 [==============================] - 0s 1ms/step - loss: 0.1638
Epoch 5/50
16/16 [==============================] - 0s 1ms/step - loss: 0.1286
Epoch 6/50
16/16 [==============================] - 0s 1ms/step - loss: 0.1024
Epoch 7/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0816
Epoch 8/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0705
Epoch 9/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0637
Epoch 10/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0595
Epoch 11/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0587
Epoch 12/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0552
Epoch 13/50
16/16 [==============================] - 0s 1m

...

[ 22:28:15] getting statistics
--------------------
training window 15768
Epoch 1/50
16/16 [==============================] - 1s 906us/step - loss: 0.2459
Epoch 2/50
16/16 [==============================] - 0s 889us/step - loss: 0.2045
Epoch 3/50
16/16 [==============================] - 0s 896us/step - loss: 0.1698
Epoch 4/50
16/16 [==============================] - 0s 870us/step - loss: 0.1322
Epoch 5/50
16/16 [==============================] - 0s 856us/step - loss: 0.1014
Epoch 6/50
16/16 [==============================] - 0s 878us/step - loss: 0.0807
Epoch 7/50
16/16 [==============================] - 0s 849us/step - loss: 0.0678
Epoch 8/50
16/16 [==============================] - 0s 867us/step - loss: 0.0618
Epoch 9/50
16/16 [==============================] - 0s 889us/step - loss: 0.0581
Epoch 10/50
16/16 [==============================] - 0s 879us/step - loss: 0.0574
Epoch 11/50
16/16 [==============================] - 0s 887us/step - loss: 0.0556
Epoch 12/50
16/16 [==============

16/16 [==============================] - 1s 788us/step - loss: 0.3848
Epoch 2/50
16/16 [==============================] - 0s 788us/step - loss: 0.3035
Epoch 3/50
16/16 [==============================] - 0s 800us/step - loss: 0.2549
Epoch 4/50
16/16 [==============================] - 0s 1ms/step - loss: 0.2016
Epoch 5/50
16/16 [==============================] - 0s 815us/step - loss: 0.1531
Epoch 6/50
16/16 [==============================] - 0s 832us/step - loss: 0.1151
Epoch 7/50
16/16 [==============================] - 0s 935us/step - loss: 0.0910
Epoch 8/50
16/16 [==============================] - 0s 983us/step - loss: 0.0770
Epoch 9/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0676
Epoch 10/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0609
Epoch 11/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0566
Epoch 12/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0526
Epoch 13/50
16/16 [===========================

/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/models/multivariate/mvfts.py:165: RuntimeWarning: invalid value encountered in double_scalars
  ret.append(np.dot(mv,mp.T)/np.nansum(mv))


[ 22:28:25] getting statistics
--------------------
training window 17739
Epoch 1/50
16/16 [==============================] - 1s 1ms/step - loss: 0.2794
Epoch 2/50
16/16 [==============================] - 0s 1ms/step - loss: 0.2341
Epoch 3/50
16/16 [==============================] - 0s 997us/step - loss: 0.1898
Epoch 4/50
16/16 [==============================] - 0s 966us/step - loss: 0.1488
Epoch 5/50
16/16 [==============================] - 0s 972us/step - loss: 0.1144
Epoch 6/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0908
Epoch 7/50
16/16 [==============================] - 0s 949us/step - loss: 0.0733
Epoch 8/50
16/16 [==============================] - 0s 985us/step - loss: 0.0656
Epoch 9/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0625
Epoch 10/50
16/16 [==============================] - 0s 2ms/step - loss: 0.0577
Epoch 11/50
16/16 [==============================] - 0s 2ms/step - loss: 0.0598
Epoch 12/50
16/16 [==========================

16/16 [==============================] - 0s 1ms/step - loss: 0.2819
Epoch 2/50
16/16 [==============================] - 0s 1ms/step - loss: 0.2394
Epoch 3/50
16/16 [==============================] - 0s 1ms/step - loss: 0.1962
Epoch 4/50
16/16 [==============================] - 0s 1ms/step - loss: 0.1479
Epoch 5/50
16/16 [==============================] - 0s 1ms/step - loss: 0.1099
Epoch 6/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0848
Epoch 7/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0689
Epoch 8/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0602
Epoch 9/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0548
Epoch 10/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0515
Epoch 11/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0489
Epoch 12/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0482
Epoch 13/50
16/16 [==============================] - 0s 1ms/

In [ ]:
#fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15,3])
#plt.plot(appliances[0][400:410].values)
#plt.plot(forecasts[0][400:410])

In [6]:
measures

,window,rmse,mape,smape
0,0,3.640910,1.226331,0.600302
1,657,1.735596,0.235708,0.117544
2,1314,1.894720,1.282029,0.636338
3,1971,2.045937,1.057183,0.566960
4,2628,7.310786,4.066821,2.176825
5,3285,3.551314,3.727053,1.721552
6,3942,12.007337,3.113522,1.454783
7,4599,15.526223,5.231075,1.845052
8,5256,6.043172,4.025617,1.890341
9,5913,8.002165,2.167615,0.854579


In [8]:
print("Mean: ")
print(statistics.mean(measures['rmse']))
print(statistics.mean(measures['mape']))
print(statistics.mean(measures['smape']))

Mean: 
9.625270493572119
3.266257890520395
1.2894553925206542
